In [1]:
"""
Notebook version of the new clustering algorithm
"""

'\nNotebook version of the new clustering algorithm\n'

In [2]:
from yuval_module.paper_clusterer import PaperClusterer
from yuval_module.paper_source import PaperSource
import sys
import pandas as pd
import numpy as np


%matplotlib inline

In [3]:
def load_dataset(set_name):
    ps=PaperSource()
    ps.load_dataset(set_name)
    return ps

In [4]:
def get_res_papers(ps,author_name):
    df=ps.get_dataset()
    return df[df['last_author_name']==author_name]

In [5]:
ps=load_dataset('enriched_labeled')

In [6]:
df=get_res_papers(ps,'Szeszko, PR')

In [7]:
df

,Unnamed: 0,pmid,PI_IDS,last_author_name,doi,journal_name,language,last_author,last_author_country,last_author_email,last_author_id,last_author_inst,last_author_inst_type,mesh,mesh_major,pub_year,title
19516,20662,19145224,1988051,"Szeszko, PR",NaN,Neuropsychopharmacology : official publication...,eng,NaN,United States,NaN,1954110.0,Zucker Hillside Hospital,Healthcare,"['Adult', 'Anisotropy', 'Diffusion Magnetic Re...","['Bipolar Disorder/pathology', 'Cerebellum/pat...",2009.0,A voxel-based diffusion tensor imaging study o...
19518,20664,18570200,1988051,"Szeszko, PR",10.1002/hbm.20595,Human brain mapping,eng,NaN,United States,NaN,1954110.0,Zucker Hillside Hospital,Healthcare,"['Adult', 'Analysis of Variance', 'Brain Mappi...","['Schizophrenia/pathology', 'Thalamus/pathology']",2009.0,Volumetric and shape analysis of the thalamus ...
19525,20671,29744804,1988051,"Szeszko, PR",NaN,Brain imaging and behavior,eng,{'affiliation': 'James J. Peters VA Medical Ce...,United States,NaN,1654036.0,James J Peters VA Medical Center,Healthcare,"['Adult', 'Brief Psychiatric Rating Scale', 'D...","['Antipsychotic Agents/therapeutic use', 'Fron...",2019.0,Frontal lobe functioning during a simple respo...
19526,20672,28348381,1988051,"Szeszko, PR",NaN,Molecular psychiatry,eng,{'affiliation': 'James J. Peters VA Medical Ce...,United States,NaN,NaN,James J Peters VA Medical Center,Healthcare,"['Adult', 'Antipsychotic Agents/therapeutic us...",['Diffusion Magnetic Resonance Imaging/methods...,2018.0,Greater extracellular free-water in first-epis...
19528,20674,25567423,1988051,"Szeszko, PR",NaN,Neuropsychopharmacology : official publication...,eng,{'affiliation': '1] Center for Psychiatric Neu...,United States,NaN,1954110.0,Zucker Hillside Hospital,Healthcare,"['Adolescent', 'Adult', 'Antipsychotic Agents'...","['Cerebral Cortex/blood supply', 'Rest']",2015.0,Abnormal resting state FMRI activity predicts ...
19531,20677,25765593,1988051,"Szeszko, PR",NaN,Progress in neuro-psychopharmacology & biologi...,eng,{'affiliation': 'Feinstein Institute for Medic...,United States,NaN,1954110.0,Zucker Hillside Hospital,Healthcare,"['Adolescent', 'Brain/blood supply', 'Case-Con...","['Autistic Disorder/pathology', 'Brain Mapping...",2015.0,Neural systems mediating decision-making and r...
19532,20678,25081550,1988051,"Szeszko, PR",NaN,Brain structure & function,eng,{'affiliation': 'Center for Psychiatric Neuros...,United States,NaN,1550922.0,Albert Einstein College of Medicine,Education,"['Adult', 'Diffusion Magnetic Resonance Imagin...",['Hippocampus/anatomy & histology'],2015.0,Delineation of hippocampal subregions using T1...
19533,20679,25532972,1988051,"Szeszko, PR",NaN,Bipolar disorders,eng,{'affiliation': 'The Feinstein Institute for M...,United States,NaN,1710365.0,Northwell Health,Healthcare,"['Adolescent', 'Adult', 'Age Factors', 'Child'...","['Bipolar Disorder/diagnosis', 'Corpus Callosu...",2015.0,Age-associated alterations in corpus callosum ...


In [8]:
def get_clusters(author):
    author_df = get_res_papers(author)
    paper_clusterer=PaperClusterer(eps=1.27)
    res_clusters, cluster_dfs=paper_clusterer.get_dist_matrix(author_df)
    return cluster_dfs[0].sort_values(["cluster"])

In [9]:
def get_out_path(row):
    cur_name=row["last_author_name"]
    fields=cur_name.split(", ")
    return "/home/ubuntu/data/{}_{}.csv".format(fields[0], fields[1])

In [10]:
# def get_file_name(author_name):
#     author_name=author_name.replace(',',' ')
#     author_rebuilt='_'.join(author_name.split(' '))
#     return ''.join(["/home/ubuntu/data/", author_rebuilt, ".csv"])
# #%%
def main(author_list):
    print(author_list)
    
    for author in author_list:
        print(author)
        cluster_df=get_clusters(author)
        # out_path=get_out_path(row)
        # cluster_df.to_csv(out_path, index=False, columns=["pmid", "cluster", "last_author_inst", "last_author_forename","pub_year","last_author_name"])
        
#         #pmid_df=pd.read_csv(list_path, dtype=str, usecols=["PMID"]).rename(columns={"PMID":"Highlight"})
#         #print(pmid_df.info())
#         #limit=min(limit, len(pmid_df))
#         #pmid_list=pmid_df.Highlight.tolist()[offset:offset+limit]
#         pmid_list=pmid_df.pmid.tolist()[:2]
#         paper_clusterer=PaperClusterer(eps=1.27)
#         res_clusters, cluster_dfs=paper_clusterer.infer_paper_author_data(pmid_list)
    
#     #res_clusters, cluster_dfs=paper_clusterer.infer_paper_author_data(pmid_list).sort_values(["rownum"])
#         return res_clusters, cluster_dfs
    #out_path="/home/ubuntu/data/rp_40000_authors.tsv"
    #out_path="/home/ubuntu/data/test_forenames_res.tsv"

    #res_clusters.to_csv(out_path, index=False, sep='\t')


In [12]:
author_list=['Szeszko, PR']
main(author_list)

['Szeszko, PR']
Szeszko, PR


NameError: name 'get_papers' is not defined

In [ ]:
print(cluster_dfs[0].cluster.value_counts())

In [ ]:
out_path="/home/ubuntu/data/hecht_jr.csv"
#out_path="/home/ubuntu/data/gottlieb_pa.csv"
cluster_dfs[0].sort_values(["cluster"]).to_csv(out_path, index=False, columns=["pmid", "cluster", "last_author_inst", "last_author_forename","pub_year","last_author_name"])

In [ ]:
# cur_clusters=cluster_dfs[0]
# print(cur_clusters.info())
# print(cur_clusters.cluster.value_counts())
# cur_clusters.to_csv("/home/ubuntu/data/friedman_n.csv", index=False)

In [ ]:
# cur_clusters=cluster_dfs[0][["pmid", "last_author_id", "last_author_inst", 
#                              "last_author_country", "last_author_name",
#                              "last_author_forename", "cluster",
#                             "pub_year", "last_author_affiliation"]].sort_values(["cluster"])
# print(cur_clusters)

In [ ]:
# cm = 'tab20b'
# cur_clusters=cluster_dfs[20][["cluster","pmid","last_author_inst",  "last_author_forename",
#                              "mesh_clean", "other_authors", "last_author_country", "last_author_name"]].reset_index()
# cur_clusters.sort_values('cluster').style.background_gradient(cmap=cm, subset=['cluster']).set_caption('Clustering output')

In [ ]:
def forename_delta(n1, n2):
        if ' ' in n1:
            n1=n1.split()[0]
        if ' ' in n2:
            n2=n2.split()[0]
        if n1==n2:
            if len(n1)>1:
                return 1.0
            else:
                return 0.5
        elif n1 in n2 or n2 in n1:
            return 0.75
        else:
            return 0.0

In [ ]:
forename_delta("Peter A","P A")